In [3]:
import os

entries = os.listdir('./Camera')

In [13]:

path='./Camera/'

In [15]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np

class Img2Vec():

    def __init__(self, cuda=False, model='resnet-18', layer='default', layer_output_size=512):
        """ Img2Vec
        :param cuda: If set to True, will run forward pass on GPU
        :param model: String name of requested model
        :param layer: String or Int depending on model.  See more docs: https://github.com/christiansafka/img2vec.git
        :param layer_output_size: Int depicting the output size of the requested layer
        """
        self.device = torch.device("cuda" if cuda else "cpu")
        self.layer_output_size = layer_output_size
        self.model_name = model
        
        self.model, self.extraction_layer = self._get_model_and_layer(model, layer)

        self.model = self.model.to(self.device)

        self.model.eval()

        self.scaler = transforms.Scale((224, 224))
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
        self.to_tensor = transforms.ToTensor()

    def get_vec(self, img, tensor=False):
        """ Get vector embedding from PIL image
        :param img: PIL Image or list of PIL Images
        :param tensor: If True, get_vec will return a FloatTensor instead of Numpy array
        :returns: Numpy ndarray
        """
        if type(img) == list:
            a = [self.normalize(self.to_tensor(self.scaler(im))) for im in img]
            images = torch.stack(a).to(self.device) 
            if self.model_name == 'alexnet':
                my_embedding = torch.zeros(len(img), self.layer_output_size)
            else:
                my_embedding = torch.zeros(len(img), self.layer_output_size, 1, 1)

            def copy_data(m, i, o):
                my_embedding.copy_(o.data)

            h = self.extraction_layer.register_forward_hook(copy_data)
            h_x = self.model(images)
            h.remove()

            if tensor:
                return my_embedding
            else:
                if self.model_name == 'alexnet':
                    return my_embedding.numpy()[:, :]
                else:
                    print(my_embedding.numpy()[:, :, 0, 0].shape)
                    return my_embedding.numpy()[:, :, 0, 0]
        else:
            image = self.normalize(self.to_tensor(self.scaler(img))).unsqueeze(0).to(self.device)

            if self.model_name == 'alexnet':
                my_embedding = torch.zeros(1, self.layer_output_size)
            else:
                my_embedding = torch.zeros(1, self.layer_output_size, 1, 1)

            def copy_data(m, i, o):
                my_embedding.copy_(o.data)

            h = self.extraction_layer.register_forward_hook(copy_data)
            h_x = self.model(image)
            h.remove()

            if tensor:
                return my_embedding
            else:
                if self.model_name == 'alexnet':
                    return my_embedding.numpy()[0, :]
                else:
                    return my_embedding.numpy()[0, :, 0, 0]

    def _get_model_and_layer(self, model_name, layer):
        """ Internal method for getting layer from model
        :param model_name: model name such as 'resnet-18'
        :param layer: layer as a string for resnet-18 or int for alexnet
        :returns: pytorch model, selected layer
        """
        if model_name == 'resnet-18':
            model = models.resnet18(pretrained=True)
            if layer == 'default':
                layer = model._modules.get('avgpool')
                self.layer_output_size = 512
            else:
                layer = model._modules.get(layer)

            return model, layer

        elif model_name == 'alexnet':
            model = models.alexnet(pretrained=True)
            if layer == 'default':
                layer = model.classifier[-2]
                self.layer_output_size = 4096
            else:
                layer = model.classifier[-layer]

            return model, layer

        else:
            raise KeyError('Model %s was not found' % model_name)



In [16]:
import pdb
import os

input_path=path
entries = os.listdir('./Camera')
# entries

In [19]:
img2vec = Img2Vec()
from PIL import Image
# For each test image, we store the filename and vector as key, value in a dictionary
pics = {}
i=0
for file in os.listdir('./Camera'):
    filename = os.fsdecode(file)
    img = Image.open(os.path.join('./Camera', filename))
    vec = img2vec.get_vec(img)
    pics[filename] = vec    
    i=i+1
    if(i%200==0):
        print(i)



200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400


OSError: cannot identify image file './Camera\\VID_20160827_071713.mp4'

In [20]:
i

3506

In [21]:
import pickle
with open("My_rotation_matrix.bin", "wb") as output:
    pickle.dump(pics, output)

In [22]:
import pickle
with open("My_rotation_matrix.bin", "rb") as data:
    pics = pickle.load(data)

In [23]:
vector_key=[]
for key in pics:
  # print(key,pics[key])
  vector_key+=[pics[key]]
  # break

In [24]:
len(vector_key)

3506

In [25]:
n_clusters=5
from sklearn.cluster import KMeans
Kmean = KMeans(n_clusters=5)
Kmean.fit(vector_key)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [26]:
Kmean.labels_

array([4, 3, 1, ..., 3, 3, 3])

In [28]:
n_clusters=5 

# Python program to explain os.mkdir() method 
  
# importing os module 
import os 
  
# Directory 
# directory = "1"
  
# Parent Directory path 

# os.mkdir(path)   
# Path 
for i in range(0,5):
    path='./Camera'
    path = os.path.join(path, str(i)) 
    os.mkdir(path) 

In [29]:

Kmean.labels_[2]

1

In [30]:
i=0;
count=0;
for key in pics:
    if((pics[key]==vector_key[i]).all()):
        count+=1;
    i+=1;
print(i,count)

3506 3506


In [32]:
from PIL import Image
i=0;
count=0;
for key in pics:
    path='./Camera/'
    path = path+'/'+str(Kmean.labels_[i])
    img = Image.open(os.path.join('./Camera/', key))
    img = img.save(path+'/'+key) 
    i+=1;
# print(i,count)for i in range(0,len(Kmean.labels_)):
#     path = os.path.join(path, str(Kmean.labels_[i])) 
#     # os.mkdir(path) 
#     im1 = im1.save(path+) 

In [33]:
i

3506